# 💾 LangGraph Avanzado - Persistencia y Human-in-the-Loop
## Parte 2: Checkpoints y Aprobaciones

### 🎯 Objetivos:

1. **Persistencia con Checkpoints** - Guardar y reanudar grafos
2. **Human-in-the-Loop** (HITL) - Aprobaciones y validaciones
3. **Integración con PostgreSQL** - Guardar estado en base de datos
4. **Manejo de errores y retry** - Grafos robustos para producción
5. **Streaming de respuestas** - Respuestas en tiempo real

---

## 📦 Setup

In [1]:
!pip install langgraph langchain-openai langchain-core python-dotenv psycopg2-binary -q

print("✅ Paquetes instalados")

✅ Paquetes instalados


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv('OPENAI_API_KEY'):
    raise ValueError("⚠️ OPENAI_API_KEY no encontrada")

print("✅ Configuración lista")

✅ Configuración lista


---
# 💾 PARTE 1: Persistencia con Checkpoints

## ¿Qué son los Checkpoints?

Los **checkpoints** permiten:
- ✅ **Pausar** un grafo y continuar después
- ✅ **Time-travel**: volver a estados anteriores
- ✅ **Recovery**: recuperarse de errores
- ✅ **Auditoría**: ver todo el historial de ejecución
- ✅ **A/B testing**: comparar diferentes caminos

### Caso de uso:

```
Conversación larga:
  Usuario: "Necesito cotización"
    → Agente: "¿Qué equipo?"
       [CHECKPOINT #1]
  Usuario: "Demoledor"
    → Agente: "¿Cuántos días?"
       [CHECKPOINT #2]
  [Usuario cierra pestaña] ❌
  
  [Usuario vuelve al día siguiente]
    → Sistema carga CHECKPOINT #2
    → Continúa desde donde quedó ✅
```

## Tipos de Checkpointers:

1. **MemorySaver** - En memoria (solo para desarrollo)
2. **SqliteSaver** - SQLite (local, simple)
3. **PostgresSaver** - PostgreSQL (producción, distribuido)
4. **RedisSaver** - Redis (ultra-rápido, caché)

## 1.1 Ejemplo Simple: MemorySaver

In [3]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from operator import add

# State con reducer para mensajes
class StateConversacion(TypedDict):
    messages: Annotated[list, add]  # Se van agregando mensajes
    paso_actual: int
    informacion_recolectada: dict

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Nodos de un flujo de cotización
def preguntar_equipo(state: StateConversacion) -> StateConversacion:
    print("\n📝 Paso 1: Preguntando por el equipo...")
    return {
        "messages": [HumanMessage(content="¿Qué equipo necesitas rentar?")],
        "paso_actual": 1,
        "informacion_recolectada": state.get("informacion_recolectada", {})
    }

def procesar_equipo(state: StateConversacion) -> StateConversacion:
    print("\n📝 Paso 2: Procesando equipo...")
    # Última respuesta del usuario
    ultimo_msg = state["messages"][-1].content if state["messages"] else "No especificado"
    
    info = state.get("informacion_recolectada", {})
    info["equipo"] = ultimo_msg
    
    return {
        "messages": [HumanMessage(content="¿Por cuántos días lo necesitas?")],
        "paso_actual": 2,
        "informacion_recolectada": info
    }

def procesar_dias(state: StateConversacion) -> StateConversacion:
    print("\n📝 Paso 3: Procesando días...")
    ultimo_msg = state["messages"][-1].content if state["messages"] else "No especificado"
    
    info = state.get("informacion_recolectada", {})
    info["dias"] = ultimo_msg
    
    # Generar cotización con LLM
    prompt = f"""Genera una cotización breve para:
    Equipo: {info.get('equipo', 'No especificado')}
    Días: {info.get('dias', 'No especificado')}
    
    Incluye precio estimado (inventa uno realista) y descuentos si aplica."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    return {
        "messages": [HumanMessage(content=f"Cotización:\n{response.content}")],
        "paso_actual": 3,
        "informacion_recolectada": info
    }

# Construir grafo
workflow = StateGraph(StateConversacion)

workflow.add_node("preguntar_equipo", preguntar_equipo)
workflow.add_node("procesar_equipo", procesar_equipo)
workflow.add_node("procesar_dias", procesar_dias)

workflow.add_edge(START, "preguntar_equipo")
workflow.add_edge("preguntar_equipo", "procesar_equipo")
workflow.add_edge("procesar_equipo", "procesar_dias")
workflow.add_edge("procesar_dias", END)

# 🔑 CLAVE: Compilar con checkpointer
memory = MemorySaver()
app_con_memoria = workflow.compile(checkpointer=memory)

print("✅ Grafo con persistencia creado")
print("\n💡 Ahora cada ejecución se guarda como checkpoint")

✅ Grafo con persistencia creado

💡 Ahora cada ejecución se guarda como checkpoint


## 1.2 Ejecutar con Checkpoints

In [4]:
print("="*80)
print("🎬 DEMO: CONVERSACIÓN CON CHECKPOINTS")
print("="*80)

# 🔑 IMPORTANTE: Necesitas un thread_id para identificar la conversación
config = {"configurable": {"thread_id": "cotizacion-123"}}

# Paso 1: Preguntar por equipo
print("\n👤 Usuario inicia conversación")
resultado = app_con_memoria.invoke(
    {"messages": [], "paso_actual": 0, "informacion_recolectada": {}},
    config
)
print(f"🤖 Sistema: {resultado['messages'][-1].content}")
print(f"📊 Paso actual: {resultado['paso_actual']}")

# Paso 2: Usuario responde
print("\n👤 Usuario: 'Demoledor eléctrico'")
resultado = app_con_memoria.invoke(
    {"messages": [HumanMessage(content="Demoledor eléctrico")], "paso_actual": resultado['paso_actual'], "informacion_recolectada": resultado['informacion_recolectada']},
    config
)
print(f"🤖 Sistema: {resultado['messages'][-1].content}")
print(f"📊 Paso actual: {resultado['paso_actual']}")
print(f"📦 Info guardada: {resultado['informacion_recolectada']}")

print("\n⏸️ SIMULANDO: Usuario cierra pestaña / se desconecta")
print("💾 El estado está guardado en memoria...")

print("\n" + "─"*80)
print("⏰ SIMULANDO: Usuario vuelve al día siguiente")
print("─"*80)

# Paso 3: Usuario vuelve y continúa (MISMO thread_id)
print("\n👤 Usuario: '15 días'")
resultado = app_con_memoria.invoke(
    {"messages": [HumanMessage(content="15 días")], "paso_actual": resultado['paso_actual'], "informacion_recolectada": resultado['informacion_recolectada']},
    config
)
print(f"\n🤖 Sistema: {resultado['messages'][-1].content}")
print(f"\n✅ Conversación completada!")
print(f"📦 Información final recolectada: {resultado['informacion_recolectada']}")

print("\n" + "="*80)
print("💡 OBSERVA:")
print("   - La conversación se mantuvo entre múltiples invocaciones")
print("   - El state se preservó automáticamente")
print("   - Se puede pausar y continuar en cualquier momento")
print("="*80)

🎬 DEMO: CONVERSACIÓN CON CHECKPOINTS

👤 Usuario inicia conversación

📝 Paso 1: Preguntando por el equipo...

📝 Paso 2: Procesando equipo...

📝 Paso 3: Procesando días...
🤖 Sistema: Cotización:
**Cotización Breve para Renta de Equipo**

**Equipo:**
- Proyector Full HD
- Pantalla de proyección (100 pulgadas)
- Sistema de sonido portátil

**Días:** 3 días

**Precio Estimado:**
- Proyector Full HD: $150/día
- Pantalla de proyección: $50/día
- Sistema de sonido portátil: $75/día

**Subtotal:**
- Proyector: $450
- Pantalla: $150
- Sonido: $225

**Total sin descuento:** $825

**Descuento aplicado:** 10% por renta de más de 3 días

**Total con descuento:** $742.50

---

**Nota:** Los precios son estimados y pueden variar según la disponibilidad y condiciones específicas. Para formalizar la renta, se requiere un depósito del 20%. 

Si necesitas más información o deseas realizar la renta, no dudes en contactarnos. ¡Estamos aquí para ayudarte!
📊 Paso actual: 3

👤 Usuario: 'Demoledor eléctrico'

📝

## 1.3 Inspeccionar Checkpoints

In [5]:
print("\n🔍 EXPLORANDO CHECKPOINTS GUARDADOS")
print("="*80)

# Obtener historial de checkpoints
checkpoints = list(app_con_memoria.get_state_history(config))

print(f"\n📊 Total de checkpoints: {len(checkpoints)}")
print("\n💾 Historial completo de la conversación:\n")

for i, checkpoint in enumerate(checkpoints, 1):
    print(f"\n{'-'*80}")
    print(f"Checkpoint #{i}")
    print(f"{'-'*80}")
    print(f"ID: {checkpoint.config['configurable']['checkpoint_id'][:8]}...")
    print(f"Paso: {checkpoint.values.get('paso_actual', 'N/A')}")
    print(f"Info recolectada: {checkpoint.values.get('informacion_recolectada', {})}")
    if checkpoint.values.get('messages'):
        ultimo_msg = checkpoint.values['messages'][-1].content[:100]
        print(f"Último mensaje: {ultimo_msg}...")

print("\n" + "="*80)
print("💡 Puedes hacer 'time-travel' a cualquier checkpoint")
print("="*80)


🔍 EXPLORANDO CHECKPOINTS GUARDADOS

📊 Total de checkpoints: 15

💾 Historial completo de la conversación:


--------------------------------------------------------------------------------
Checkpoint #1
--------------------------------------------------------------------------------
ID: 1f0ddb1b...
Paso: 3
Info recolectada: {'equipo': '¿Qué equipo necesitas rentar?', 'dias': '¿Por cuántos días lo necesitas?'}
Último mensaje: Cotización:
**Cotización de Renta de Equipo**

**Equipo solicitado:** Proyector HD  
**Días de renta...

--------------------------------------------------------------------------------
Checkpoint #2
--------------------------------------------------------------------------------
ID: 1f0ddb1b...
Paso: 2
Info recolectada: {'equipo': '¿Qué equipo necesitas rentar?', 'dias': '¿Por cuántos días lo necesitas?'}
Último mensaje: ¿Por cuántos días lo necesitas?...

--------------------------------------------------------------------------------
Checkpoint #3
--------------

## 1.4 Time Travel - Volver a un Checkpoint Anterior

In [6]:
print("\n⏰ TIME TRAVEL: Volviendo al checkpoint anterior")
print("="*80)

# Obtener el penúltimo checkpoint
checkpoints = list(app_con_memoria.get_state_history(config))
if len(checkpoints) >= 2:
    checkpoint_anterior = checkpoints[1]  # [0] es el último, [1] es el anterior
    
    print(f"\n📍 Checkpoint actual: Paso {checkpoints[0].values['paso_actual']}")
    print(f"⏮️  Volviendo a: Paso {checkpoint_anterior.values['paso_actual']}")
    
    # Continuar desde ese checkpoint
    config_antiguo = checkpoint_anterior.config
    
    # Podemos modificar la respuesta
    print("\n👤 Usuario cambia de opinión: 'Mejor 30 días para descuento'")
    resultado_alternativo = app_con_memoria.invoke(
        {"messages": [HumanMessage(content="Mejor 30 días para descuento")], "paso_actual": checkpoint_anterior.values['paso_actual'], "informacion_recolectada": checkpoint_anterior.values['informacion_recolectada']},
        config  # Mismo thread
    )
    
    print(f"\n🤖 Nueva cotización: {resultado_alternativo['messages'][-1].content}")
    print("\n✅ Time-travel completado - nueva rama creada")
else:
    print("⚠️ No hay suficientes checkpoints para time-travel")


⏰ TIME TRAVEL: Volviendo al checkpoint anterior

📍 Checkpoint actual: Paso 3
⏮️  Volviendo a: Paso 2

👤 Usuario cambia de opinión: 'Mejor 30 días para descuento'

📝 Paso 1: Preguntando por el equipo...

📝 Paso 2: Procesando equipo...

📝 Paso 3: Procesando días...

🤖 Nueva cotización: Cotización:
**Cotización de Renta de Equipo**

**Equipo requerido:** Proyector HD  
**Cantidad:** 1  
**Días de renta:** 3  

**Precio estimado:** $150.00 por día  
**Subtotal:** $450.00  

**Descuento por renta de 3 días:** 10%  
**Descuento aplicado:** -$45.00  

**Total a pagar:** $405.00  

*Nota: El precio incluye accesorios básicos y entrega en la ubicación solicitada. Para confirmar la reserva, se requiere un depósito del 30% del total.* 

¡Gracias por considerar nuestra oferta! Si tienes alguna pregunta o necesitas más información, no dudes en contactarnos.

✅ Time-travel completado - nueva rama creada


---
# 🙋 PARTE 2: Human-in-the-Loop (HITL)

## ¿Qué es HITL?

**Human-in-the-Loop** permite:
- ⏸️ **Pausar** la ejecución y esperar aprobación humana
- ✅ **Validar** decisiones críticas antes de ejecutarlas
- ✏️ **Editar** el state antes de continuar
- 🔄 **Redirigir** el flujo según decisión humana

## Casos de uso:

```
1. Cotizaciones grandes:
   Sistema calcula → Gerente aprueba → Se envía

2. Decisiones legales:
   IA analiza contrato → Abogado revisa → Se firma

3. Operaciones de riesgo:
   Agente propone acción → Humano valida → Se ejecuta

4. Control de calidad:
   IA genera respuesta → QA revisa → Se publica
```

## 2.1 Grafo con Aprobación Humana

In [ ]:
from typing import Literal

class StateAprobacion(TypedDict):
    cliente: str
    monto_cotizacion: float
    descuento_propuesto: float
    requiere_aprobacion: bool
    aprobado: bool
    comentario_gerente: str
    estado_final: str

# Umbrales de aprobación
UMBRAL_AUTO_APROBACION = 5000  # < HNL 5,000 se aprueba automáticamente
UMBRAL_GERENTE = 20000  # < HNL 20,000 requiere gerente
# > HNL 20,000 requiere director

def calcular_cotizacion(state: StateAprobacion) -> StateAprobacion:
    print(f"\n💰 Calculando cotización para {state['cliente']}...")
    print(f"   Monto: HNL {state['monto_cotizacion']:,.2f}")
    print(f"   Descuento propuesto: {state['descuento_propuesto']*100}%")
    
    # Determinar si requiere aprobación
    if state['monto_cotizacion'] < UMBRAL_AUTO_APROBACION:
        requiere = False
        print("   ✅ Monto bajo - Aprobación automática")
    else:
        requiere = True
        nivel = "Gerente" if state['monto_cotizacion'] < UMBRAL_GERENTE else "Director"
        print(f"   ⚠️ Requiere aprobación de: {nivel}")
    
    return {**state, "requiere_aprobacion": requiere}

def aprobar_automatico(state: StateAprobacion) -> StateAprobacion:
    print("\n✅ APROBACIÓN AUTOMÁTICA")
    print("   Cotización procesada y enviada al cliente")
    
    return {
        **state,
        "aprobado": True,
        "estado_final": "Aprobado automáticamente"
    }

def solicitar_aprobacion(state: StateAprobacion) -> StateAprobacion:
    print("\n⏸️ PAUSADO - REQUIERE APROBACIÓN HUMANA")
    print("="*80)
    print(f"Cliente: {state['cliente']}")
    print(f"Monto: HNL {state['monto_cotizacion']:,.2f}")
    print(f"Descuento: {state['descuento_propuesto']*100}%")
    print(f"Total con descuento: HNL {state['monto_cotizacion'] * (1 - state['descuento_propuesto']):,.2f}")
    print("="*80)
    print("\n⏳ Esperando decisión del gerente...")
    
    # En un sistema real, aquí se enviaría una notificación
    # y el flujo se pausaría hasta recibir respuesta
    
    return state

def procesar_decision(state: StateAprobacion) -> StateAprobacion:
    if state['aprobado']:
        print("\n✅ APROBADO POR GERENTE")
        print(f"   Comentario: {state.get('comentario_gerente', 'Sin comentarios')}")
        estado = "Aprobado por gerente"
    else:
        print("\n❌ RECHAZADO POR GERENTE")
        print(f"   Razón: {state.get('comentario_gerente', 'Sin comentarios')}")
        estado = "Rechazado"
    
    return {**state, "estado_final": estado}

# Routing
def decidir_flujo(state: StateAprobacion) -> Literal["aprobar_auto", "solicitar_aprobacion"]:
    if state['requiere_aprobacion']:
        return "solicitar_aprobacion"
    else:
        return "aprobar_auto"

# Construir grafo
workflow_aprobacion = StateGraph(StateAprobacion)

workflow_aprobacion.add_node("calcular", calcular_cotizacion)
workflow_aprobacion.add_node("aprobar_auto", aprobar_automatico)
workflow_aprobacion.add_node("solicitar_aprobacion", solicitar_aprobacion)
workflow_aprobacion.add_node("procesar_decision", procesar_decision)

workflow_aprobacion.add_edge(START, "calcular")

workflow_aprobacion.add_conditional_edges(
    "calcular",
    decidir_flujo,
    {
        "aprobar_auto": "aprobar_auto",
        "solicitar_aprobacion": "solicitar_aprobacion"
    }
)

workflow_aprobacion.add_edge("aprobar_auto", END)

# 🔑 IMPORTANTE: solicitar_aprobacion NO tiene edge a END
# Se queda "colgando" hasta que se invoque de nuevo con la decisión
workflow_aprobacion.add_edge("solicitar_aprobacion", "procesar_decision")
workflow_aprobacion.add_edge("procesar_decision", END)

# Compilar con memoria
memory_aprobacion = MemorySaver()
app_aprobacion = workflow_aprobacion.compile(
    checkpointer=memory_aprobacion,
    interrupt_before=["procesar_decision"]  # 🔑 PAUSA antes de este nodo
)

print("✅ Grafo con HITL creado")
print("\n🔑 interrupt_before=['procesar_decision'] pausará la ejecución")

## 2.2 🎬 DEMO: Flujo con Aprobación

In [8]:
print("="*80)
print("🎬 DEMO 1: Cotización pequeña (aprobación automática)")
print("="*80)

config1 = {"configurable": {"thread_id": "cotiz-auto-001"}}

resultado = app_aprobacion.invoke(
    {
        "cliente": "Constructora ABC",
        "monto_cotizacion": 3500.00,
        "descuento_propuesto": 0.10,
        "requiere_aprobacion": False,
        "aprobado": False,
        "comentario_gerente": "",
        "estado_final": ""
    },
    config1
)

print(f"\n📊 Estado final: {resultado['estado_final']}")
print(f"✅ Flujo completado sin intervención humana")

🎬 DEMO 1: Cotización pequeña (aprobación automática)

💰 Calculando cotización para Constructora ABC...
   Monto: $3,500.00
   Descuento propuesto: 10.0%
   ✅ Monto bajo - Aprobación automática

✅ APROBACIÓN AUTOMÁTICA
   Cotización procesada y enviada al cliente

📊 Estado final: Aprobado automáticamente
✅ Flujo completado sin intervención humana


In [9]:
print("\n\n" + "="*80)
print("🎬 DEMO 2: Cotización grande (requiere aprobación)")
print("="*80)

config2 = {"configurable": {"thread_id": "cotiz-gerente-002"}}

# Primera ejecución - se pausará
print("\n🚀 Iniciando flujo...")
resultado = app_aprobacion.invoke(
    {
        "cliente": "Mega Construcciones XYZ",
        "monto_cotizacion": 15000.00,
        "descuento_propuesto": 0.20,
        "requiere_aprobacion": False,
        "aprobado": False,
        "comentario_gerente": "",
        "estado_final": ""
    },
    config2
)

# Verificar estado
estado_actual = app_aprobacion.get_state(config2)
print(f"\n📍 Nodo actual: {estado_actual.next}")
print("💡 El grafo está PAUSADO esperando aprobación")

print("\n" + "─"*80)
print("⏰ SIMULANDO: Gerente revisa y aprueba al día siguiente")
print("─"*80)

# Actualizar el state con la decisión del gerente
app_aprobacion.update_state(
    config2,
    {
        "aprobado": True,
        "comentario_gerente": "Aprobado. Cliente frecuente, buen historial de pago."
    }
)

print("\n✍️ Gerente actualizó la decisión en el sistema")
print("\n🚀 Continuando flujo...")

# Continuar ejecución
resultado_final = app_aprobacion.invoke(None, config2)

print(f"\n📊 Estado final: {resultado_final['estado_final']}")
print("✅ Flujo completado con aprobación humana")



🎬 DEMO 2: Cotización grande (requiere aprobación)

🚀 Iniciando flujo...

💰 Calculando cotización para Mega Construcciones XYZ...
   Monto: $15,000.00
   Descuento propuesto: 20.0%
   ⚠️ Requiere aprobación de: Gerente

⏸️ PAUSADO - REQUIERE APROBACIÓN HUMANA
Cliente: Mega Construcciones XYZ
Monto: $15,000.00
Descuento: 20.0%
Total con descuento: $12,000.00

⏳ Esperando decisión del gerente...

📍 Nodo actual: ('procesar_decision',)
💡 El grafo está PAUSADO esperando aprobación

────────────────────────────────────────────────────────────────────────────────
⏰ SIMULANDO: Gerente revisa y aprueba al día siguiente
────────────────────────────────────────────────────────────────────────────────

✍️ Gerente actualizó la decisión en el sistema

🚀 Continuando flujo...

✅ APROBADO POR GERENTE
   Comentario: Aprobado. Cliente frecuente, buen historial de pago.

📊 Estado final: Aprobado por gerente
✅ Flujo completado con aprobación humana


In [10]:
print("\n\n" + "="*80)
print("🎬 DEMO 3: Cotización rechazada")
print("="*80)

config3 = {"configurable": {"thread_id": "cotiz-rechazada-003"}}

# Primera ejecución
resultado = app_aprobacion.invoke(
    {
        "cliente": "Empresa Nueva Inc",
        "monto_cotizacion": 25000.00,
        "descuento_propuesto": 0.25,  # 25% de descuento - muy alto
        "requiere_aprobacion": False,
        "aprobado": False,
        "comentario_gerente": "",
        "estado_final": ""
    },
    config3
)

print("\n⏳ Gerente revisa...")

# Gerente rechaza
app_aprobacion.update_state(
    config3,
    {
        "aprobado": False,
        "comentario_gerente": "Descuento muy alto para cliente nuevo. Reducir a 15% máximo."
    }
)

# Continuar
resultado_final = app_aprobacion.invoke(None, config3)

print(f"\n📊 Estado final: {resultado_final['estado_final']}")
print("\n💡 En un sistema real, esto dispararía:")
print("   - Notificación al vendedor")
print("   - Solicitud de recotizar con nuevo descuento")
print("   - Log de auditoría")



🎬 DEMO 3: Cotización rechazada

💰 Calculando cotización para Empresa Nueva Inc...
   Monto: $25,000.00
   Descuento propuesto: 25.0%
   ⚠️ Requiere aprobación de: Director

⏸️ PAUSADO - REQUIERE APROBACIÓN HUMANA
Cliente: Empresa Nueva Inc
Monto: $25,000.00
Descuento: 25.0%
Total con descuento: $18,750.00

⏳ Esperando decisión del gerente...

⏳ Gerente revisa...

❌ RECHAZADO POR GERENTE
   Razón: Descuento muy alto para cliente nuevo. Reducir a 15% máximo.

📊 Estado final: Rechazado

💡 En un sistema real, esto dispararía:
   - Notificación al vendedor
   - Solicitud de recotizar con nuevo descuento
   - Log de auditoría


---
# 📚 PARTE 3: Mejores Prácticas y Patrones

## ✅ Patrones de Persistencia

### 1. Conversaciones largas
```python
# Usar thread_id único por conversación
thread_id = f"user-{user_id}-session-{timestamp}"
config = {"configurable": {"thread_id": thread_id}}
```

### 2. Multi-tenant (múltiples clientes)
```python
# Incluir tenant en el thread_id
thread_id = f"tenant-{tenant_id}-user-{user_id}-{session_id}"
```

### 3. A/B Testing
```python
# Guardar checkpoints antes de decisiones críticas
checkpoint_id = app.get_state(config).config['configurable']['checkpoint_id']

# Probar camino A
resultado_a = app.invoke({...}, config)

# Volver y probar camino B
config_b = {**config, "checkpoint_id": checkpoint_id}
resultado_b = app.invoke({...}, config_b)
```

## ✅ Patrones de HITL

### 1. Aprobación simple
```python
# Pausar antes de nodo crítico
app = workflow.compile(
    checkpointer=memory,
    interrupt_before=["ejecutar_accion"]
)
```

### 2. Aprobación multi-nivel
```python
# Diferentes niveles según el monto
if monto < 10000:
    interrupt_before = ["supervisor"]
elif monto < 50000:
    interrupt_before = ["gerente"]
else:
    interrupt_before = ["director"]
```

### 3. Edición del state
```python
# Humano puede modificar el state antes de continuar
app.update_state(config, {
    "descuento": 0.15,  # Reducir descuento
    "plazo_dias": 30     # Cambiar plazo
})
```

## ⚠️ Errores Comunes

### ❌ No usar thread_id único
```python
# MAL - Todos los usuarios comparten el mismo state
config = {"configurable": {"thread_id": "default"}}

# BIEN - Thread único por conversación
config = {"configurable": {"thread_id": f"user-{user_id}-{uuid4()}"}}
```

### ❌ No limpiar checkpoints antiguos
```python
# Implementar limpieza periódica
# Ver init-db.sql para función cleanup_old_checkpoints()
```

### ❌ No manejar timeouts en HITL
```python
# Agregar timeout para aprobaciones
created_at = app.get_state(config).created_at
if (datetime.now() - created_at) > timedelta(hours=24):
    # Auto-rechazar o escalar
    app.update_state(config, {"aprobado": False, "timeout": True})
```

---

# 🎯 Ejercicios Propuestos

## Ejercicio 1: Sistema de tickets con escalamiento
Crea un grafo que:
1. Recibe un ticket de soporte
2. Lo clasifica (bajo, medio, alto, crítico)
3. Tickets críticos van a gerente
4. Si no se responde en 1 hora, escala a director
5. Guarda todo en checkpoints para auditoría

## Ejercicio 2: Workflow de publicación de contenido
Implementa:
1. Autor escribe borrador → CHECKPOINT
2. Editor revisa → Aprueba/Rechaza
3. Si rechaza, vuelve a autor (time-travel)
4. Si aprueba, va a QA
5. QA revisa formato → Aprueba/Rechaza
6. Si OK, publica

## Ejercicio 3: Integrar con PostgreSQL
Modifica los ejemplos para:
1. Usar PostgresSaver en lugar de MemorySaver
2. Guardar métricas en la tabla usage_metrics
3. Implementar caché en response_cache
4. Generar reporte de checkpoints guardados

---

## 📖 Recursos

- [LangGraph Persistence](https://langchain-ai.github.io/langgraph/concepts/persistence/)
- [Human-in-the-Loop](https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/)
- [Checkpointers](https://langchain-ai.github.io/langgraph/reference/checkpoints/)

---

**¡Excelente trabajo! Ahora dominas persistencia y HITL en LangGraph** 🎉